In [2]:
#!pip install pycep_correios geopy pandas sqlalchemy python-dotenv

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
# conexão com o banco de dados SQL Server
engine = create_engine(
  os.environ.get('DATABASE_URL_SQL_SERVER'),
  connect_args = {"TrustServerCertificate": "yes"},
  echo=False
)

# conexão postgres
engine_postgres = create_engine(
  os.environ.get('DATABASE_URL_SQL_POSTGRES'),
  echo=False
)

# conexão mariadb/mysql
engine_mysql = create_engine(
  os.environ.get('DATABASE_URL_SQL_MYSQL'),
  echo=False
)

In [2]:
file_path = os.path.join('Postos de Saúde.xlsx')
df = pd.read_excel(file_path, engine='openpyxl')
#file_path = os.path.join('postos_saude_brasil.csv')
#df = pd.read_csv(file_path)
df.head()

,UF,MUNICÍPIO,ESTABELECIMENTO,LOGRADOURO,NÚMERO,BAIRRO
0,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA,RUA 07 QUEDAS,284,CENTRO
1,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA JOAO DANIEL DAMASCENO,BR 364 KM 114,114,ZONA RURAL
2,AC,ACRELANDIA,UNIDADE MISTA DE SAUDE DE ACRELANDIA,AVENIDA PARANA,346,CENTRO
3,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA CICERO BATISTA,BR 364 KM 90 RAMAL GRANADA KM 16,16,ZONA RURAL
4,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA REDENCAO,AC 475 VILA REDENCAOI,NaN,ZONA RURAL


In [4]:
# renomeia as colunas para tirar o acento
a_renomear = {
    'MUNICÍPIO': 'MUNICIPIO',
    'NÚMERO': 'NUMERO'
}
df.rename(columns=a_renomear, inplace=True)

for column in df.columns:
  df[column] = df[column].astype(str).str.strip()

# cria o campo endereço completo para facilitar a chamada à API
df['ENDERECO_COMPLETO'] = df['LOGRADOURO'] + ', ' + df['BAIRRO'] + ', ' + df['MUNICIPIO'] + ', ' + df['UF'] + ', ' + 'BRASIL'
# inclui novas colunas
df['latitude'] = None
df['longitude'] = None
df['address'] = None
df['point'] = None

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46821 entries, 0 to 46820
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   UF                 46821 non-null  object
 1   MUNICIPIO          46821 non-null  object
 2   ESTABELECIMENTO    46821 non-null  object
 3   LOGRADOURO         46821 non-null  object
 4   NUMERO             46821 non-null  object
 5   BAIRRO             46821 non-null  object
 6   ENDERECO_COMPLETO  46821 non-null  object
 7   latitude           0 non-null      object
 8   longitude          0 non-null      object
 9   address            0 non-null      object
 10  point              0 non-null      object
dtypes: object(11)
memory usage: 3.9+ MB


In [5]:
# salva arquivo CSV para facilitar novas consultas
df.to_csv('postos_saude_brasil_completo_raw.csv', index=False)

In [8]:
# importa para a base de dados
nome_tabela = 'unidades_vacinacao'
df.to_sql(
    nome_tabela,
    #engine, 
    #engine_postgres,
    engine_mysql,
    if_exists='replace',
    chunksize=500,
    #schema='dbo'
)
print('Registros salvos com sucesso')

Registros salvos com sucesso
